In [66]:
import pandas as pd
from catboost import CatBoostRegressor

1. Загружаем тестовые данные

In [67]:
test_df = pd.read_excel("data/excel/test.xlsx")

2. Загружаем обученную модель

In [68]:
model = CatBoostRegressor()
model.load_model("catboost_model.cbm")

3. Подготавливаем признаки

In [69]:
# Преобразование даты
test_df["dt"] = pd.to_datetime(test_df["dt"], dayfirst=True)
test_df["year"] = test_df["dt"].dt.year
test_df["month"] = test_df["dt"].dt.month
test_df["day"] = test_df["dt"].dt.day
test_df["week"] = test_df["dt"].dt.isocalendar().week

for i in range(1, 7):
    test_df[f"lag_{i}"] = test_df["Price"].shift(i)

# Добавление столбца Price_source
test_df["Price_source"] = test_df["Price"].shift(2)

4. Выбираем признаки и делаем прогноз

In [70]:
# Подготовка данных для предсказания
feature_columns = model.feature_names_
X_test = test_df[feature_columns].dropna()

# Предсказание
test_df.loc[X_test.index, "Прогноз_цены"] = model.predict(X_test)

5. Определяем N недель закупки

In [71]:
# Определение закупки
def calculate_weeks(row):
    if row["Прогноз_цены"] > row["Price"] * 1.02:
        return 6
    elif row["Прогноз_цены"] > row["Price"]:
        return 4
    elif row["Прогноз_цены"] < row["Price"] * 0.98:
        return 1
    else:
        return 3

test_df["Закупка_недель"] = test_df.apply(calculate_weeks, axis=1)

6. Сохраняем результаты

In [72]:
test_df.to_excel("predicted_procurement.xlsx", index=False)
print("Прогнозирование завершено. Результаты сохранены в predicted_procurement.xlsx")

Прогнозирование завершено. Результаты сохранены в predicted_procurement.xlsx
